In [53]:
import polars as pl

In [55]:
metadata = pl.read_csv("/zata/zippy/ramirezc/gembs_smk/.test2/?type=Experiment&%40id=%2Fexperiments%2FENCSR442FIY%2F&files.output_type=reads&files.output_category=raw+data", separator="\t")

In [3]:
metadata.head()

File accession,File format,File type,File format type,Output type,File assembly,Experiment accession,Assay,Donor(s),Biosample term id,Biosample term name,Biosample type,Biosample organism,Biosample treatments,Biosample treatments amount,Biosample treatments duration,Biosample genetic modifications methods,Biosample genetic modifications categories,Biosample genetic modifications targets,Biosample genetic modifications gene targets,Biosample genetic modifications site coordinates,Biosample genetic modifications zygosity,Experiment target,Library made from,Library depleted in,Library extraction method,Library lysis method,Library crosslinking method,Library strand specific,Experiment date released,Project,RBNS protein concentration,Library fragmentation method,Library size range,Biological replicate(s),Technical replicate(s),Read length,Mapped read length,Run type,Paired end,Paired with,Index of,Derived from,Size,Lab,md5sum,dbxrefs,File download URL,Genome annotation,Platform,Controlled by,File Status,s3_uri,Azure URL,File analysis title,File analysis status,Audit WARNING,Audit NOT_COMPLIANT,Audit ERROR
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,i64,str,str,i64,str,str,str,i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
"""ENCFF413KHN""","""fastq""","""fastq""",null,"""reads""",null,"""ENCSR765JPC""","""WGBS""","""/human-donors/ENCDO000AAD/""","""EFO:0002067""","""K562""","""cell line""","""Homo sapiens""",null,null,null,null,null,null,null,null,null,null,"""DNA""",null,null,null,null,null,"""2016-03-22""","""ENCODE""",null,"""shearing (Covaris generic)""","""100-300""",1,"""1_1""",100,null,"""paired-ended""",1,"""/files/ENCFF567DAI/""",null,null,82296261270,"""Richard Myers, HAIB""","""10f907ee59a2f5bb84630fd918257e…","""SRA:SRR4235744""","""https://www.encodeproject.org/…",null,"""Illumina HiSeq 2500""",null,"""released""","""s3://encode-public/2015/07/06/…","""https://datasetencode.blob.cor…",null,null,"""mixed read lengths, low covera…","""insufficient coverage""","""missing lambda C conversion ra…"
"""ENCFF567DAI""","""fastq""","""fastq""",null,"""reads""",null,"""ENCSR765JPC""","""WGBS""","""/human-donors/ENCDO000AAD/""","""EFO:0002067""","""K562""","""cell line""","""Homo sapiens""",null,null,null,null,null,null,null,null,null,null,"""DNA""",null,null,null,null,null,"""2016-03-22""","""ENCODE""",null,"""shearing (Covaris generic)""","""100-300""",1,"""1_1""",100,null,"""paired-ended""",2,"""/files/ENCFF413KHN/""",null,null,79247018841,"""Richard Myers, HAIB""","""362c730ce3439d9ea8ed158dafdd98…","""SRA:SRR4235744""","""https://www.encodeproject.org/…",null,"""Illumina HiSeq 2500""",null,"""released""","""s3://encode-public/2015/07/21/…","""https://datasetencode.blob.cor…",null,null,"""mixed read lengths, low covera…","""insufficient coverage""","""missing lambda C conversion ra…"
"""ENCFF336KJH""","""fastq""","""fastq""",null,"""reads""",null,"""ENCSR765JPC""","""WGBS""","""/human-donors/ENCDO000AAD/""","""EFO:0002067""","""K562""","""cell line""","""Homo sapiens""",null,null,null,null,null,null,null,null,null,null,"""DNA""",null,null,null,null,null,"""2016-03-22""","""ENCODE""",null,"""shearing (Covaris generic)""","""100-300""",2,"""2_1""",125,null,"""paired-ended""",1,"""/files/ENCFF585HYM/""",null,null,43193181424,"""Richard Myers, HAIB""","""be6dfb07ae352214eb5454dcfcfd5c…","""SRA:SRR4235743""","""https://www.encodeproject.org/…",null,"""Illumina HiSeq 2500""",null,"""released""","""s3://encode-public/2015/12/17/…","""https://datasetencode.blob.cor…",null,null,"""mixed read lengths, low covera…","""insufficient coverage""","""missing lambda C conversion ra…"
"""ENCFF585HYM""","""fastq""","""fastq""",null,"""reads""",null,"""ENCSR765JPC""","""WGBS""","""/human-donors/ENCDO000AAD/""","""EFO:0002067""","""K562""","""cell line""","""Homo sapiens""",null,null,null,null,null,null,null,null,null,null,"""DNA""

In [58]:
slim_metadata = metadata.select("File accession", "File download URL", "Biological replicate(s)", "Technical replicate(s)", "Run type", "Paired end", "Paired with")

In [59]:
with_paths = metadata.with_columns(
   ("/zata/zippy/ramirezc/gembs_smk/.test2/" + pl.col("File accession") + ".fastq.gz").alias("File path"),
   pl.col("Paired with").str.split("/").list[-2]
)
slim_metadata = with_paths.select("File accession", "Experiment accession", "File path", "Biological replicate(s)", "Technical replicate(s)", "Run type", "Paired end", "Paired with")
slim_metadata.write_csv("/zata/zippy/ramirezc/gembs_smk/.test2/metadata.tsv", separator="\t")


In [ ]:
slim_metadata = pl.read_csv("metadata.tsv", separator="\t")

# # Add one if single-ended
# add_one = slim_metadata.with_columns(
#     pl.when(pl.col("Run type") == "single-ended").then(
#         pl.lit(1).alias("Paired end")
#     ).otherwise(
#         pl.col("Paired end")
#     )
# )
# add_one.head()

grouped_read = slim_metadata.group_by(
        "Biological replicate(s)", "Technical replicate(s)"
    ).agg(
        "File path", "File accession", "Experiment accession"
    ).with_columns([
        pl.when(pl.col("File accession").list.len() == 1)
        .then(pl.col("File path").list.get(0))
        .alias("File"),
    
        pl.when(pl.col("File accession").list.len() == 2)
        .then(pl.col("File path").list.get(0))
        .alias("File1"),
    
        pl.when(pl.col("File accession").list.len() == 2)
        .then(pl.col("File path").list.get(1))
        .alias("File2"),
        
        pl.col("Experiment accession").list.first().alias("Dataset"),
        pl.col("File accession").list.join("_").alias("Name")
    ]).with_columns(
        pl.col("Name").hash(seed=33).alias("Barcode")
    )
grouped_read.head()

if grouped_read["File"].has_nulls:
    gembs_metadata = grouped_read.select(
        "Barcode", "Name", "Dataset", "File1", "File2"
    )
elif grouped_read["File1"].has_nulls and grouped_read["File2"].has_nulls:
    gembs_metadata = grouped_read.select(
        "Barcode", "Name", "Dataset", "File"
    )
gembs_metadata.head()


grouped_read =  metadata.filter(pl.col("Paired end") == 1).with_columns([
    pl.col("File accession").hash(seed=33).alias("Barcode"),
    pl.col("File accession").alias("Name"),
    pl.col("Experiment accession").alias("Dataset"),
    pl.col("File path").alias("File1")
]).join(slim_metadata.filter(pl.col("Paired end") == 2).select("Paired with", pl.col("File path").alias("File2")),
    left_on="File accession",
    right_on="Paired with",
    how="left"
).select("Barcode", "Name", "Dataset", "File1", "File2")
grouped_read

.rename({
    ""
}).write_csv(output.csv, sep="\t")

Barcode,Name,Dataset,File1,File2
u64,str,str,str,str
12200173259172864394,"""ENCFF413KHN_ENCFF567DAI""","""ENCSR765JPC""","""/zata/zippy/ramirezc/gembs_smk…","""/zata/zippy/ramirezc/gembs_smk…"
14098407479695328579,"""ENCFF336KJH_ENCFF585HYM""","""ENCSR765JPC""","""/zata/zippy/ramirezc/gembs_smk…","""/zata/zippy/ramirezc/gembs_smk…"


In [4]:
# Test regex parsing
import os

before = "results/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta.gz"
after = os.path.basename(before)
print(before)
print(after)

import re

pattern = r"\.(fa|fasta|fa\.gz|fasta\.gz)$"

test_files = [
    "GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta.gz",
    "reference.fasta",
    "genome.fa.gz", 
    "sample.fa",
    "data.txt"  # Invalid extension
]

for filename in test_files:
    basename = re.sub(pattern, '', filename)
    print(f"{filename} -> {basename}")

results/GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta.gz
GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta.gz
GRCh38_no_alt_analysis_set_GCA_000001405.15.fasta.gz -> GRCh38_no_alt_analysis_set_GCA_000001405.15
reference.fasta -> reference
genome.fa.gz -> genome
sample.fa -> sample
data.txt -> data.txt


In [ ]:
num_list = ['1', '1', '2']
i

In [50]:
import json
from bs4 import BeautifulSoup
import pandas as pd
from io import StringIO
from collections import defaultdict

table_headings = [
    "Mapping Stats (Reads)",
    "Uniquely Mapping Fragments (MAPQ >= 20)",
    "Mapping Stats (Reads)",
    "Bisulfite Conversion Rate",
    "Correct Pairs",
    "Read Lengths",
    "Mismatch Distribution",
]

with open("/zata/zippy/ramirezc/gembs_smk/results/report/mapping/14098407479695328579/14098407479695328579.html", 'r') as f:
    html_content = f.read()

soup = BeautifulSoup(html_content, 'html.parser')
tables_dict = {}

for h1 in soup.find_all('h1'):
    header_text = h1.get_text(strip=True)
    
    if header_text in table_headings:
        next_table = h1.find_next_sibling('table')
        
        if next_table:
            try:
                table_html = StringIO(next_table.prettify())
                df_list = pd.read_html(table_html)
                
                if df_list:
                    df = df_list[0]
                    tables_dict[header_text] = df
                    print(f"Extracted: {header_text} ({df.shape})")
                    
            except Exception as e:
                print(f"Error parsing '{header_text}': {e}")

Extracted: Mapping Stats (Reads) ((5, 7))
Extracted: Uniquely Mapping Fragments (MAPQ >= 20) ((2, 2))
Extracted: Mapping Stats (Reads) ((5, 7))
Extracted: Bisulfite Conversion Rate ((2, 2))
Extracted: Correct Pairs ((1, 2))
Extracted: Read Lengths ((2, 7))
Extracted: Mismatch Distribution ((26, 7))


In [51]:
print(tables_dict)

{'Mapping Stats (Reads)':   Base  Total Bases        %     Pair One      %.1     Pair Two      %.2
0    A  55533696670  39.20 %  21499697098  30.36 %  34033999572  48.05 %
1    C  14741526650  10.41 %    928270609   1.31 %  13813256041  19.50 %
2    G  15081376305  10.65 %  13764110705  19.43 %   1317265600   1.86 %
3    T  55082066622  38.89 %  34061856734  48.09 %  21020209888  29.68 %
4    N   1214177065   0.86 %    572486510   0.81 %    641690555   0.91 %, 'Uniquely Mapping Fragments (MAPQ >= 20)':             Concept      Value
0  Unique Fragments  414022243
1          % Unique    75.07 %, 'Bisulfite Conversion Rate':                      Bisulfite Conversion Type  Conversion Rate
0  Conversion Rate of non-methylated Cytosines              NaN
1      Conversion Rate of methylated Cytosines              NaN, 'Correct Pairs':          Concept  Read Pairs
0  Correct Pairs   470180178, 'Read Lengths':    Read Length  Total Reads        %   Read One      %.1   Read Two      %.2
0      

In [ ]:
import polars as pl

slim_metadata = pl.read_csv("/zata/zippy/ramirezc/gembs_smk/.test2/metadata.tsv", separator="\t")

grouped_read = slim_metadata.group_by(
        "Biological replicate(s)", "Technical replicate(s)", maintain_order=True
    ).agg(
        "File path", "File accession", "Experiment accession"
    ).with_columns([
        pl.when(pl.col("File path").cast(pl.List(pl.Utf8)).list.len() == 1)
        .then(pl.col("File path").list.get(0))
        .alias("File"),
    
        pl.when(pl.col("File path").cast(pl.List(pl.Utf8)).list.len() == 2)
        .then(pl.col("File path").list.get(0))
        .alias("File1"),
    
        pl.when(pl.col("File path").cast(pl.List(pl.Utf8)).list.len() == 2)
        .then(pl.col("File path").list.get(1))
        .alias("File2"),
        
        (pl.col("Experiment accession").list.first() + pl.lit("_") + pl.col("Biological replicate(s)").cast(pl.String)).alias("Dataset"),
        pl.col("File accession").list.join("_").alias("Name")
    ]).with_columns(
        pl.col("Name").hash(seed=33).alias("Barcode")
    )

if grouped_read["File"].has_nulls:
    gembs_metadata = grouped_read.select(
        "Barcode", "Name", "Dataset", "File1", "File2"
    ).cast(pl.String)
elif grouped_read["File1"].has_nulls and grouped_read["File2"].has_nulls:
    gembs_metadata = grouped_read.select(
        "Barcode", "Name", "Dataset", "File"
    ).cast(pl.String)

print(gembs_metadata)

# gembs_metadata.write_csv(output.csv)

shape: (4, 5)
┌──────────────────────┬─────────────┬───────────────┬───────┬───────┐
│ Barcode              ┆ Name        ┆ Dataset       ┆ File1 ┆ File2 │
│ ---                  ┆ ---         ┆ ---           ┆ ---   ┆ ---   │
│ str                  ┆ str         ┆ str           ┆ str   ┆ str   │
╞══════════════════════╪═════════════╪═══════════════╪═══════╪═══════╡
│ 9758964677840593144  ┆ ENCFF340YVI ┆ ENCSR442FIY_2 ┆ null  ┆ null  │
│ 15142513135076508350 ┆ ENCFF703DEV ┆ ENCSR442FIY_1 ┆ null  ┆ null  │
│ 2943539977550304610  ┆ ENCFF802SFU ┆ ENCSR442FIY_1 ┆ null  ┆ null  │
│ 18413901726945152902 ┆ ENCFF306ZPW ┆ ENCSR442FIY_2 ┆ null  ┆ null  │
└──────────────────────┴─────────────┴───────────────┴───────┴───────┘
